In [1]:

from elftools.elf.elffile import ELFFile
from elftools.dwarf.descriptions import (
    describe_DWARF_expr, set_global_machine_arch)
from elftools.dwarf.locationlists import (
    LocationEntry, LocationExpr, LocationParser)







In [2]:
def get_DIE_at_offset(CU, offset):
        for die in CU.iter_DIEs():
            if die.offset == CU.cu_offset+offset:
                return die 
        return None

def get_type_name(CU, offset):#get_DIE_at_offset(CU,attr.value)
    die = get_DIE_at_offset(CU, offset)
    
    if die.tag == 'DW_TAG_pointer_type':
        for _attr in die.attributes.values():
            if _attr.name== "DW_AT_type":
                return "*"+get_type_name(CU,_attr.value) 
        
    
    for attr in die.attributes.values():
        if attr.name== "DW_AT_name":
            return attr.value.decode("utf-8")
    
    


In [3]:
from collections import defaultdict
def line_entry_mapping(line_program):
    filename_map = defaultdict(int)

    # The line program, when decoded, returns a list of line program
    # entries. Each entry contains a state, which we'll use to build
    # a reverse mapping of filename -> #entries.
    lp_entries = line_program.get_entries()
    for lpe in lp_entries:
        # We skip LPEs that don't have an associated file.
        # This can happen if instructions in the compiled binary
        # don't correspond directly to any original source file.
        if not lpe.state or lpe.state.file == 0:
            continue
        filename = lpe_filename(line_program, lpe.state.file)
        filename_map[filename] += 1

    # for filename, lpe_count in filename_map.items():
    #     print("    filename=%s -> %d entries" % (filename, lpe_count))
    return filename_map

def lpe_filename(line_program, file_index):

    lp_header = line_program.header
    file_entries = lp_header["file_entry"]
    

    # File and directory indices are 1-indexed.
    file_entry = file_entries[file_index - 1]
    dir_index = file_entry["dir_index"]

    # A dir_index of 0 indicates that no absolute directory was recorded during
    # compilation; return just the basename.
    if dir_index == 0:
        return file_entry.name.decode()

    directory = lp_header["include_directory"][dir_index - 1]
    return posixpath.join(directory, file_entry.name).decode()

In [4]:
def show_loclist(loclist, dwarfinfo, indent, cu_offset):
    """ Display a location list nicely, decoding the DWARF expressions
        contained within.
    """
    d = []
    for loc_entity in loclist:
        if isinstance(loc_entity, LocationEntry):
            d.append('%s <<%s>>' % (
                loc_entity,
                describe_DWARF_expr(loc_entity.loc_expr, dwarfinfo.structs, cu_offset)))
        else:
            d.append(str(loc_entity))
    return '\n'.join(indent + s for s in d)

In [5]:
# https://github.com/eliben/pyelftools/blob/master/examples/dwarf_location_info.py


from elftools.dwarf.locationlists import LocationParser, LocationExpr
FUNC_PARAMS = {}

def process_file(filename):
    print('Processing file:', filename)
    with open(filename, 'rb') as f:
        elffile = ELFFile(f)

        if not elffile.has_dwarf_info():
            print('  file has no DWARF info')
            return

        # get_dwarf_info returns a DWARFInfo context object, which is the
        # starting point for all DWARF-based processing in pyelftools.
        dwarfinfo = elffile.get_dwarf_info()
        
        # The location lists are extracted by DWARFInfo from the .debug_loc
        # section, and returned here as a LocationLists object.
        location_lists = dwarfinfo.location_lists()

        # This is required for the descriptions module to correctly decode
        # register names contained in DWARF expressions.
        set_global_machine_arch(elffile.get_machine_arch())

        # Create a LocationParser object that parses the DIE attributes and
        # creates objects representing the actual location information.
        loc_parser = LocationParser(location_lists)
        
        section_offset = dwarfinfo.debug_info_sec.global_offset
        # Offset of the .debug_info section in the stream
        

        
        for CU in dwarfinfo.iter_CUs():
            
            #get source file name
            print(CU)
            line_program = dwarfinfo.line_program_for_CU(CU)
            print('TEST!!!')
            filename_map = line_entry_mapping(line_program)
            CU_src_file =  list(filename_map.items())[0][0]
            
            CU_dictionary_key = CU_src_file 
            
            FUNC_PARAMS[CU_dictionary_key] = {}
            
            print('  Found a compile unit at offset %s, length %s' % (
                CU.cu_offset, CU['unit_length']))

            # A CU provides a simple API to iterate over all the DIEs in it.
            die_depth = 0
            are_DIEs_of_function = False
            FUNC_name = None
            for DIE in CU.iter_DIEs():
                
                ############################################################
                #############   Prasing Function DIEs start ################
                
                print()
                

                            
                
                if DIE.tag == 'DW_TAG_subprogram':
                    are_DIEs_of_function = True
                    
                    for attr in DIE.attributes.values():
                        if attr.name == "DW_AT_name": #FUNC NAME
                            FUNC_name = attr.value.decode("utf-8")
                            FUNC_PARAMS[CU_dictionary_key][FUNC_name] ={}
                            print("SUBPROGRAM: ",FUNC_name)
                            
                if DIE.tag == 'DW_TAG_formal_parameter' or DIE.tag =='DW_TAG_variable':
                    tags = [attr.name for attr in DIE.attributes.values()]
                    PARAM_name = None
                    if "DW_AT_name" in tags:
                        for attr in DIE.attributes.values():
                            if attr.name == "DW_AT_name": # VAR NAME 
                                PARAM_name = attr.value.decode("utf-8")
                                FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {}
                                print(PARAM_name)
                            if attr.name == "DW_AT_type":
                                var_type = DIE.tag.split('_')[-1]
                                FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {'type':get_type_name(CU,attr.value) , 'kind':var_type}
                        
                            
                            
                            # Check if this attribute contains location information
                            if loc_parser.attribute_has_location(attr, CU['version']):
                                loc = loc_parser.parse_from_attribute(attr,
                                                                      CU['version'])
                                # We either get a list (in case the attribute is a
                                # reference to the .debug_loc section) or a LocationExpr
                                # object (in case the attribute itself contains location
                                # information).
                                print(CU_dictionary_key,FUNC_name,PARAM_name)
                                if isinstance(loc, LocationExpr):
#                                     print('      %s' % (
#                                         describe_DWARF_expr(loc.loc_expr,
#                                                             dwarfinfo.structs, CU.cu_offset)))
                                    print(describe_DWARF_expr(loc.loc_expr, dwarfinfo.structs, CU.cu_offset))
                                    FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name]["location"]= describe_DWARF_expr(loc.loc_expr, dwarfinfo.structs, CU.cu_offset)
                                elif isinstance(loc, list):
#                                     print(show_loclist(loc,
#                                                        dwarfinfo,
#                                                        '      ', CU.cu_offset))
                                    FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name]["location"]= show_loclist(loc,
                                                       dwarfinfo,'      ', CU.cu_offset)
                        
                        

                                
#                 if DIE.tag == 'DW_TAG_variable':
#                     PARAM_name = None
#                     for attr in DIE.attributes.values():
#                         if attr.name == "DW_AT_name": # VAR NAME 
#                             PARAM_name = attr.value.decode("utf-8")
#                             print(PARAM_name)
#                             FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {}

#                         if attr.name == "DW_AT_type":
#                             FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {'type':get_type_name(CU,attr.value) , 'kind':'variable'}

#                     print("__________________\n")

                ###############################################
                #############  parsing  Function DIEs ends ################
                


                
                if DIE.is_null(): #https://chromium.googlesource.com/chromiumos/third_party/pyelftools/+/25a77f7738d7fe824f2ed4d33a123136b9d8e88a/scripts/readelf.py
                    are_DIEs_of_function = False
                    FUNC_name = None
                    
                    die_depth -= 1
                    continue
                if DIE.has_children:
                    die_depth += 1
                    

filename = './../../binaries/c_many/stacktest'
process_file(filename)

Processing file: ./../../binaries/c_many/stacktest
TEST!!!
  Found a compile unit at offset 0, length 986























SUBPROGRAM:  free



SUBPROGRAM:  printf





SUBPROGRAM:  malloc



SUBPROGRAM:  fake

a
stack.c fake a
(DW_OP_fbreg: -36)

b
stack.c fake b
(DW_OP_fbreg: -40)

x
stack.c fake x
(DW_OP_fbreg: -44)

d
stack.c fake d
(DW_OP_fbreg: -32)

e
stack.c fake e
(DW_OP_fbreg: -28)

f
stack.c fake f
(DW_OP_fbreg: -24)

g
stack.c fake g
(DW_OP_fbreg: -20)


SUBPROGRAM:  pop

stk_ptr
stack.c pop stk_ptr
(DW_OP_fbreg: -56)

int1
stack.c pop int1
(DW_OP_fbreg: -44)

int2
stack.c pop int2
(DW_OP_fbreg: -40)

number
stack.c pop number
(DW_OP_fbreg: -36)

stk
stack.c pop stk
(DW_OP_fbreg: -32)

tmp
stack.c pop tmp
(DW_OP_fbreg: -24)




SUBPROGRAM:  dummy

a_int
stack.c dummy a_int
(DW_OP_fbreg: -148)

b_float
stack.c dummy b_float
(DW_OP_fbreg: -152)

c_double
stack.c dummy c_double
(DW_OP_fbreg: -160)

char_pointer
stack.c dummy char_pointer
(DW_OP_fbreg: -168)

e_int
stack.

In [14]:
FUNC_PARAMS 


{'stack.c': {'free': {},
  'printf': {},
  'malloc': {},
  'fake': {'a': {'type': 'int',
    'kind': 'parameter',
    'location': '(DW_OP_fbreg: -36)'},
   'b': {'type': 'int', 'kind': 'parameter', 'location': '(DW_OP_fbreg: -40)'},
   'x': {'type': 'int', 'kind': 'parameter', 'location': '(DW_OP_fbreg: -44)'},
   'd': {'type': 'int', 'kind': 'variable', 'location': '(DW_OP_fbreg: -32)'},
   'e': {'type': 'int', 'kind': 'variable', 'location': '(DW_OP_fbreg: -28)'},
   'f': {'type': 'int', 'kind': 'variable', 'location': '(DW_OP_fbreg: -24)'},
   'g': {'type': 'int', 'kind': 'variable', 'location': '(DW_OP_fbreg: -20)'}},
  'pop': {'stk_ptr': {'type': '**stack',
    'kind': 'parameter',
    'location': '(DW_OP_fbreg: -56)'},
   'int1': {'type': 'int',
    'kind': 'variable',
    'location': '(DW_OP_fbreg: -44)'},
   'int2': {'type': 'int',
    'kind': 'variable',
    'location': '(DW_OP_fbreg: -40)'},
   'number': {'type': 'int',
    'kind': 'variable',
    'location': '(DW_OP_fbreg: -

In [7]:
#!/usr/bin/env python3

import sys,os
from elftools.elf.elffile import ELFFile
from elftools.elf.segments import Segment



filePath = './../../binaries/c_many/stacktest'


In [8]:
# https://www.capstone-engine.org/lang_python.html



from capstone import *

from capstone.x86 import *


address_inst = {}
with open(filePath, 'rb') as f:
    elf = ELFFile(f)
    code = elf.get_section_by_name('.text')
    ops = code.data()
    print('code data size: ',code.data_size)
    addr = code['sh_addr']
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    for inst in md.disasm(ops, addr):
        
        address_inst[hex(inst.address)] = inst
        

        print('\n'*3)
        print(inst.address, inst.mnemonic+"  "+inst.op_str)
        (regs_read, regs_write) = inst.regs_access()

        if len(inst.operands) > 0:
            c=-1
            for o in inst.operands:
                c += 1
                if o.type == CS_OP_MEM:
                    print("\t\toperands[%u].type: MEM" %c)
                    if o.value.mem.base != 0:
                        print("\t\t\toperands[%u].mem.base: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.base)))
                    if o.value.mem.index != 0:
                        print("\t\t\toperands[%u].mem.index: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.index)))
                    if o.value.mem.disp != 0:
                        print("\t\t\toperands[%u].mem.disp: 0x%x" \
                            %(c, o.value.mem.disp))
                    print(hex(o.value.mem.disp),o.value.mem.disp)



code data size:  1336




4288 endbr64  




4292 xor  ebp, ebp




4294 mov  r9, rdx




4297 pop  rsi




4298 mov  rdx, rsp




4301 and  rsp, 0xfffffffffffffff0




4305 push  rax




4306 push  rsp




4307 xor  r8d, r8d




4310 xor  ecx, ecx




4312 lea  rdi, [rip + 0x2b2]
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0x2b2
0x2b2 690




4319 call  qword ptr [rip + 0x2ef3]
		operands[0].type: MEM
			operands[0].mem.base: REG = rip
			operands[0].mem.disp: 0x2ef3
0x2ef3 12019




4325 hlt  




4326 nop  word ptr cs:[rax + rax]
		operands[0].type: MEM
			operands[0].mem.base: REG = rax
			operands[0].mem.index: REG = rax
0x0 0




4336 lea  rdi, [rip + 0x2f19]
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0x2f19
0x2f19 12057




4343 lea  rax, [rip + 0x2f12]
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0x2f12
0x2f12 12050




4350 cmp  rax, rdi




4353 je  0x1118



In [9]:
from collections import defaultdict
import posixpath


In [10]:
addr_lineProgram ={}
addr_sourceFile = {}
with open(filePath, 'rb') as f:
    elffile = ELFFile(f)

    if not elffile.has_dwarf_info():
        print('  file has no DWARF info')
        exit(0)

    dwarfinfo = elffile.get_dwarf_info()
    for CU in dwarfinfo.iter_CUs():
        print('  Found a compile unit at offset %s, length %s' % (
            CU.cu_offset, CU['unit_length']))

        # Every compilation unit in the DWARF information may or may not
        # have a corresponding line program in .debug_line.
        line_program = dwarfinfo.line_program_for_CU(CU)
        if line_program is None:
            print('  DWARF info is missing a line program for this CU')
            continue

        # Print a reverse mapping of filename -> #entries
        filename_map = line_entry_mapping(line_program)
        for line_entry in line_program.get_entries():

            if line_entry.state!=None:
                addr_lineProgram[hex(line_entry.state.address)] = line_entry
                addr_sourceFile [hex(line_entry.state.address)] = filename_map

        
        


  Found a compile unit at offset 0, length 986
  Found a compile unit at offset 990, length 568
  Found a compile unit at offset 1562, length 484


In [11]:
# print(addr_lineProgram)
# address_inst

def getSource(sourceFileName, row , col):
    basePath = "/home/nahid/reverse/binaries/c_many/"
    sourceFilePath = os.path.join(basePath , sourceFileName)
    sourceFile = open(sourceFilePath, "r")
    fileContent = sourceFile.readlines()
    
    row_content =  fileContent[row-1]
    row_content = row_content[:(col-1)] + '|'+row_content[(col-1)]+'|' +row_content[col:]
    
    return row_content


In [12]:

# FUNCTION_DECL
# https://stackoverflow.com/questions/43460605/function-boundary-identification-using-libclang
# https://eli.thegreenplace.net/2011/07/03/parsing-c-in-python-with-clang


import clang.cindex



def get_all_var_types(source_path):
    srcFileName = source_path.split('/')[-1]

    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.VAR_DECL:
            # print(dir(f))
            print(f.extent.start.file.name)
            originFileName = f.extent.start.file.name.split('/')[-1]
            print(originFileName)
            print(f.displayname)
            print(f.type.spelling)
            print(f.extent.start.line,f.extent.start.column)
            print('\n')
            
            

   

def get_all_function_types(source_path):
    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.FUNCTION_DECL:
            # print(dir(f))
            print(f.displayname)
            print('function name: ',( f.spelling))
            print('Returns: ',(f.result_type.spelling))
            
            
            arg_len = len(list(f.type.argument_types()))
            if arg_len>0:
                arg_types = list(f.type.argument_types())
                for arg_type in arg_types:
                    print('arg_type:',arg_type.spelling)
                args = list(f.get_arguments())
                for arg in args:
                    print('arg:',arg.spelling)


            print("\n\n\n")

            
def get_function_boundaries(source_path):
    
    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.FUNCTION_DECL:

            function_name = f.displayname.split('(')[0]
            function_boundary_by_name[function_name]={}
            function_boundary_by_name[function_name] = { 'src_path':f.extent.start.file.name,
                              'src_file':f.extent.start.file.name.split('/')[-1],
                              'start_line':f.extent.start.line,
                              'start_col':f.extent.start.column,
                              'end_line':f.extent.end.line,
                              'end_col':f.extent.end.column}
    return function_boundary_by_name

def get_containing_function(source_file_path, line, col=0):
    function_boundary_by_name = get_function_boundaries(source_file_path)
    
    for function_name, item in function_boundary_by_name.items():
        if item['src_path'] == source_file_path:
            if line>= item['start_line'] and line<= item['end_line']:
                return function_name
        

# _=get_all_var_types('/home/nahid/reverse/binaries/c_many/stack.c' )
# print(_)

_=get_all_function_types('/home/nahid/reverse/binaries/c_many/stack.c' )
print(_)

remove(const char *)
function name:  remove
Returns:  int
arg_type: const char *
arg: __filename




rename(const char *, const char *)
function name:  rename
Returns:  int
arg_type: const char *
arg_type: const char *
arg: __old
arg: __new




renameat(int, const char *, int, const char *)
function name:  renameat
Returns:  int
arg_type: int
arg_type: const char *
arg_type: int
arg_type: const char *
arg: __oldfd
arg: __old
arg: __newfd
arg: __new




fclose(FILE *)
function name:  fclose
Returns:  int
arg_type: FILE *
arg: __stream




tmpfile()
function name:  tmpfile
Returns:  FILE *




tmpnam(char *)
function name:  tmpnam
Returns:  char *
arg_type: char[20]
arg: 




tmpnam_r(char *)
function name:  tmpnam_r
Returns:  char *
arg_type: char[20]
arg: __s




tempnam(const char *, const char *)
function name:  tempnam
Returns:  char *
arg_type: const char *
arg_type: const char *
arg: __dir
arg: __pfx




fflush(FILE *)
function name:  fflush
Returns:  int
arg_type: FILE *
arg: __s

In [13]:

dir_path = './../../binaries/c_many/'

with open('stacktest.s', 'w') as outFile:
    # outFile.write('file contents\n')
    lastSource = ""
    for address in address_inst:
        inst = address_inst[address]
        instrctionCode = (address+":\t"+ inst.mnemonic+" "+inst.op_str).ljust(45)
        
        
        OFFSET = None
        if len(inst.operands) > 0:
            c=-1
            for o in inst.operands:
                c += 1
                if o.type == CS_OP_MEM:
                    print("\t\toperands[%u].type: MEM" %c)
                    if o.value.mem.base != 0:
                        print("\t\t\toperands[%u].mem.base: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.base)))
                    if o.value.mem.index != 0:
                        print("\t\t\toperands[%u].mem.index: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.index)))
                    if o.value.mem.disp != 0:
                        print("\t\t\toperands[%u].mem.disp: 0x%x" \
                            %(c, o.value.mem.disp))
                        OFFSET = o.value.mem.disp
                    print(hex(o.value.mem.disp),o.value.mem.disp)
                    
        
        if address in addr_lineProgram:
            line = addr_lineProgram[address]

            srcFileName =list(addr_sourceFile[address].keys())[0] #TODO not single file always 
            
            if srcFileName!=lastSource:
                outFile.write("\n"+ '#'*100+"\n"+ srcFileName.rjust(45) +'\n'+'#'*100+ "\n\n")
                lastSource = srcFileName
            
            sourceCode = getSource(srcFileName,addr_lineProgram[address].state.line, addr_lineProgram[address].state.column)
            function_name = get_containing_function(dir_path+srcFileName ,addr_lineProgram[address].state.line , addr_lineProgram[address].state.column)
            print(function_name)
            
            if '\n' not in  sourceCode:
                sourceCode+=sourceCode+"\n"
            outFile.write(instrctionCode+"#"+ sourceCode  )
            print(instrctionCode+"#"+ sourceCode)
            
        else:
            
            outFile.write(instrctionCode+ '\n'  )
            print(instrctionCode)
        if OFFSET:
            outFile.write("MEMORY OFFSET:     "+str(hex(OFFSET))+"     "+str(OFFSET)+'\n')
            pass

0x10c0:	endbr64                              
0x10c4:	xor ebp, ebp                         
0x10c6:	mov r9, rdx                          
0x10c9:	pop rsi                              
0x10ca:	mov rdx, rsp                         
0x10cd:	and rsp, 0xfffffffffffffff0          
0x10d1:	push rax                             
0x10d2:	push rsp                             
0x10d3:	xor r8d, r8d                         
0x10d6:	xor ecx, ecx                         
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0x2b2
0x2b2 690
0x10d8:	lea rdi, [rip + 0x2b2]               
		operands[0].type: MEM
			operands[0].mem.base: REG = rip
			operands[0].mem.disp: 0x2ef3
0x2ef3 12019
0x10df:	call qword ptr [rip + 0x2ef3]        
0x10e5:	hlt                                  
		operands[0].type: MEM
			operands[0].mem.base: REG = rax
			operands[0].mem.index: REG = rax
0x0 0
0x10e6:	nop word ptr cs:[rax + rax]          
		operands[1].type: MEM
			operands[1].mem.base: RE

dummy
0x1249:	mov dword ptr [rbp - 0x74], 2        #        int |e|_int =2;

		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0xdb0
0xdb0 3504
dummy
0x1250:	movss xmm0, dword ptr [rip + 0xdb0]  #        float |f|_float =1.0;

		operands[0].type: MEM
			operands[0].mem.base: REG = rbp
			operands[0].mem.disp: 0x-70
-0x70 -112
0x1258:	movss dword ptr [rbp - 0x70], xmm0   
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0xdab
0xdab 3499
dummy
0x125d:	movsd xmm0, qword ptr [rip + 0xdab]  #        double |g|_double = 2.0;

		operands[0].type: MEM
			operands[0].mem.base: REG = rbp
			operands[0].mem.disp: 0x-68
-0x68 -104
0x1265:	movsd qword ptr [rbp - 0x68], xmm0   
		operands[1].type: MEM
			operands[1].mem.base: REG = rbp
			operands[1].mem.disp: 0x-84
-0x84 -132
dummy
0x126a:	mov eax, dword ptr [rbp - 0x84]      #        e_int |=| a_int*2;

0x1270:	add eax, eax                         
		operands[0].type: MEM
			oper

fake
0x135d:	mov dword ptr [rbp - 0x10], 0xa      #    int |d| =10;

		operands[0].type: MEM
			operands[0].mem.base: REG = rbp
			operands[0].mem.disp: 0x-c
-0xc -12
fake
0x1364:	mov dword ptr [rbp - 0xc], 0xb       #    int |e| =11;

		operands[0].type: MEM
			operands[0].mem.base: REG = rbp
			operands[0].mem.disp: 0x-8
-0x8 -8
fake
0x136b:	mov dword ptr [rbp - 8], 0xc         #    int |f| =12;

		operands[1].type: MEM
			operands[1].mem.base: REG = rbp
			operands[1].mem.disp: 0x-14
-0x14 -20
fake
0x1372:	mov edx, dword ptr [rbp - 0x14]      #    int g = a|+|b+d+e+f;

		operands[1].type: MEM
			operands[1].mem.base: REG = rbp
			operands[1].mem.disp: 0x-18
-0x18 -24
0x1375:	mov eax, dword ptr [rbp - 0x18]      
0x1378:	add edx, eax                         
		operands[1].type: MEM
			operands[1].mem.base: REG = rbp
			operands[1].mem.disp: 0x-10
-0x10 -16
fake
0x137a:	mov eax, dword ptr [rbp - 0x10]      #    int g = a+b|+|d+e+f;

0x137d:	add edx, eax                         
		oper

main
0x1517:	mov eax, dword ptr [rbp - 0x28]      #    return |f|ake_result;

		operands[1].type: MEM
			operands[1].mem.base: REG = rbp
			operands[1].mem.disp: 0x-8
-0x8 -8
main
0x151a:	mov rdx, qword ptr [rbp - 8]         #|}||}|

		operands[1].type: MEM
			operands[1].mem.disp: 0x28
0x28 40
0x151e:	sub rdx, qword ptr fs:[0x28]         
0x1527:	je 0x152e                            
0x1529:	call 0x1090                          
0x152e:	leave                                
0x152f:	ret                                  
add
0x1530:	endbr64                              #int add(int a, int b)|{|

0x1534:	push rbp                             
0x1535:	mov rbp, rsp                         
		operands[0].type: MEM
			operands[0].mem.base: REG = rbp
			operands[0].mem.disp: 0x-14
-0x14 -20
0x1538:	mov dword ptr [rbp - 0x14], edi      
		operands[0].type: MEM
			operands[0].mem.base: REG = rbp
			operands[0].mem.disp: 0x-18
-0x18 -24
0x153b:	mov dword ptr [rbp - 0x18], esi      
		operands[0].

In [ ]:
        CU_DIR_PATH = None
        CU_FILENAME = None
        for attr in CU.get_top_DIE().attributes.values():
            if attr.name == 'DW_AT_comp_dir':
                CU_DIR_PATH = attr.value.decode("utf-8")
            if attr.name == 'DW_AT_name':
                CU_FILENAME = attr.value.decode("utf-8")